# 参考
http://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf

In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import helper.load_batch as lb
import helper.load_music as ld
import pygame
songs = ld.music_load(dir='Music_sample',length=8000)
song_test = songs[0].reshape(-1,2)
pygame.mixer.init(44100,-16,2)
sound = pygame.sndarray.make_sound(song_test)

In [3]:
import tensorflow as tf
from keras.datasets import fashion_mnist

Using TensorFlow backend.


In [4]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train, X_test = [np.reshape(_,(-1,28,28,1))/255. for _ in [X_train, X_test]]
X_val, y_val = X_train[-5000:], y_train[-5000:]
X_train, y_train = X_train[:-5000], y_train[:-5000]

In [24]:
data_dict = {
    'X_train':X_train,
    'X_val':X_val,
    'X_test':X_test,
    'y_train':y_train,
    'y_val':y_val,
    'y_test':y_test,
    'num_class':10
}

In [5]:
Winit = tf.random_normal_initializer(mean=0,stddev=0.01)
binit = tf.random_normal_initializer(mean=0.5,stddev=0.01)
Denseinit = tf.random_normal_initializer(mean=0,stddev=0.2)

In [11]:
def model(X, reg_pow=0.001):
    reg = tf.contrib.layers.l2_regularizer(scale=reg_pow)
    con = tf.constant_initializer
    with tf.variable_scope('conv1'):
        X = tf.layers.conv2d(X,32,kernel_size=[2,2],strides=[1,1],activation=tf.nn.relu
                     ,kernel_initializer=Winit,bias_initializer=binit,kernel_regularizer=reg)
    #X = tf.layers.batch_normalization(X, axis=-1, training=is_training)
        X = tf.layers.max_pooling2d(X,[2,2],2)
    with tf.variable_scope('conv2'):
        X = tf.layers.conv2d(X,128,kernel_size=[2,2],strides=[1,1],activation=tf.nn.relu
                        ,kernel_initializer=Winit,bias_initializer=binit,kernel_regularizer=reg)
        X = tf.layers.max_pooling2d(X,[2,2],2)
        X = tf.contrib.layers.flatten(X)
    with tf.variable_scope('full1'):
        X = tf.layers.dense(X,512,kernel_initializer=Denseinit,kernel_regularizer=reg,activation=tf.sigmoid)
        return X

In [12]:
def combine_predict(X1,X2):
    '''
    Input
    X1,X2: (N,M)
    y: (N,) of 1,0 value
    Return
    loss by distance
    '''
    dim = X1.get_shape()[1]
    alpha = tf.get_variable(dtype=tf.float32,shape=[dim],name='last_dense')
    diff = tf.abs(X1-X2)
    out = tf.layers.dense(diff * alpha,1,kernel_initializer=Denseinit)
    return out
    

In [28]:
tf.reset_default_graph()
X1 = tf.placeholder(tf.float32,[None,28,28,1])
X2 = tf.placeholder(tf.float32,[None,28,28,1])
y = tf.placeholder(tf.int32,[None,1])
is_training = tf.placeholder(tf.bool)

with tf.variable_scope('bottleneck') as scope:
    Y1 = model(X1)
    scope.reuse_variables()
    Y2 = model(X2)
p = combine_predict(Y1,Y2)
reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.cast(y,tf.float32),logits=p)) \
    + tf.reduce_sum(reg_losses)
optimizer = tf.train.AdamOptimizer()
train_step = optimizer.minimize(loss)


predict = tf.cast(tf.greater(p,0.), dtype=tf.int32)
correct_pred = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [29]:
data = lb.BatchLoader(**data_dict)

In [26]:
t1,t2,yt = data.make_batch(4)
t1.shape, yt.shape

((8, 28, 28, 1), (8, 1))

In [30]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [32]:
batch_size = 100
val_size = 500
for i in range(100):
    for phase in ['train','val']:
        if phase == 'train':
            step = [loss, accuracy, train_step]
            first, second, expect = data.make_batch(batch_size, dat_type='train')
        else:
            step = [loss, accuracy, correct_pred]
            first, second, expect = data.make_batch(val_size, dat_type='val')
        feed = {X1:first,X2:second,y:expect}
        current_loss, acc, _ = sess.run(step, feed_dict=feed)
        print('{} loss is {} and accuracy is {}'.format(phase, current_loss, acc))

sound.play()

train loss is 47.51131057739258 and accuracy is 0.5649999380111694
val loss is 47.13873291015625 and accuracy is 0.5689999461174011
train loss is 47.13869857788086 and accuracy is 0.5950000286102295
val loss is 46.768489837646484 and accuracy is 0.5839999914169312
train loss is 46.768463134765625 and accuracy is 0.5850000381469727
val loss is 46.40055847167969 and accuracy is 0.6119999885559082
train loss is 46.40044403076172 and accuracy is 0.6599999666213989
val loss is 46.03506088256836 and accuracy is 0.6459999680519104
train loss is 46.03520202636719 and accuracy is 0.6100000143051147
val loss is 45.67207336425781 and accuracy is 0.6309999823570251
train loss is 45.672061920166016 and accuracy is 0.6599999666213989
val loss is 45.311370849609375 and accuracy is 0.6100000143051147
train loss is 45.311363220214844 and accuracy is 0.6150000691413879
val loss is 44.953392028808594 and accuracy is 0.6109999418258667
train loss is 44.9532470703125 and accuracy is 0.6100000143051147
val 

val loss is 28.157976150512695 and accuracy is 0.7459999918937683
train loss is 28.159576416015625 and accuracy is 0.7150000333786011
val loss is 27.94144058227539 and accuracy is 0.7400000095367432
train loss is 27.912500381469727 and accuracy is 0.7599999904632568
val loss is 27.728515625 and accuracy is 0.7450000643730164
train loss is 27.701749801635742 and accuracy is 0.7800000905990601
val loss is 27.500551223754883 and accuracy is 0.7630000710487366
train loss is 27.47466278076172 and accuracy is 0.800000011920929
val loss is 27.32075309753418 and accuracy is 0.7079999446868896
train loss is 27.29220199584961 and accuracy is 0.7450000047683716
val loss is 27.086864471435547 and accuracy is 0.7590000033378601
train loss is 27.092798233032227 and accuracy is 0.7300000190734863
val loss is 26.872068405151367 and accuracy is 0.7599999904632568
train loss is 26.869232177734375 and accuracy is 0.7699999809265137
val loss is 26.667110443115234 and accuracy is 0.7899999618530273
train l

In [33]:
sound.stop()